<a href="https://colab.research.google.com/github/Spidy131/Stock-Price-Prediction-with-News-Sentiment-/blob/main/Stock_Price_Prediction_with_News_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1: Install required packages
!pip install textblob scikit-learn tensorflow newsapi-python --quiet
!pip install yfinance --upgrade --no-cache-dir

# STEP 2: Import libraries
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from textblob import TextBlob
from newsapi import NewsApiClient
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import os

# STEP 3: Define file path and download/save data if not present
file_path = 'reliance_stock.csv'
start_date = '2015-01-01'
end_date = '2025-05-08'

if not os.path.exists(file_path):
    print("📥 CSV not found. Downloading stock data...")
    df = yf.download('RELIANCE.NS', start=start_date, end=end_date)
    if df.empty or 'Close' not in df.columns:
        raise ValueError("❌ Stock data could not be loaded. Try again later.")
    df = df[['Close']].dropna().reset_index()
    df.to_csv(file_path, index=False)
    print("✅ Data downloaded and saved to reliance_stock.csv")
else:
    print("📂 CSV found. Loading saved stock data...")

# STEP 4: Load CSV
df = pd.read_csv(file_path)
print(df.head())
print(df.dtypes)
# Convert 'Close' to numeric (will turn bad values to NaN)
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

# Drop rows where 'Close' couldn't be converted (i.e., NaNs)
df = df.dropna(subset=['Close'])

# Now check
print(df.dtypes)  # 'Close' should now be float64
# STEP 5: NewsAPI sentiment (last 3 days)
newsapi = NewsApiClient(api_key='34e54dabb795492e904e598689209cad')  # Replace with your real API key

def get_sentiment_for_date(date_str):
    articles = newsapi.get_everything(
        q='Reliance Industries',
        from_param=date_str,
        to=date_str,
        language='en',
        sort_by='relevancy'
    )
    headlines = [article['title'] for article in articles['articles']]
    sentiments = [TextBlob(title).sentiment.polarity for title in headlines]
    return np.mean(sentiments) if sentiments else 0.0

sentiments = {}
for i in range(3, 0, -1):
    date = (datetime.today() - timedelta(days=i)).strftime('%Y-%m-%d')
    sentiments[date] = get_sentiment_for_date(date)

avg_sentiment = np.mean(list(sentiments.values()))
print("📰 Avg News Sentiment (Last 3 Days):", avg_sentiment)

# STEP 6: Normalize close price
scaler = MinMaxScaler()
scaled_close = scaler.fit_transform(df[['Close']])

# STEP 7: Prepare prediction input
X_input = scaled_close[-60:]
X_input = np.array([[val[0], avg_sentiment] for val in X_input])
X_input = X_input.reshape((1, 60, 2))

# STEP 8: Training data
X_data, y_data = [], []
for i in range(60, len(scaled_close)):
    price_seq = scaled_close[i-60:i]
    sentiment_seq = np.zeros((60, 1))  # Dummy sentiment for training
    combined = np.hstack((price_seq, sentiment_seq))
    X_data.append(combined)
    y_data.append(scaled_close[i][0])

X_data = np.array(X_data)
y_data = np.array(y_data)

# STEP 9: Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(60, 2)))
model.add(LSTM(50))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_data, y_data, epochs=20, batch_size=32, verbose=1)

# STEP 10: Predict tomorrow's price
predicted_scaled = model.predict(X_input)
predicted_price = scaler.inverse_transform(predicted_scaled)

print(f"\n📈 Predicted Closing Price for Tomorrow (Reliance): ₹{predicted_price[0][0]:.2f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 154.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.1 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.57
    Uninstalling yfinance-0.2.57:
      Successfully uninstalled yfinance-0.2.57
📥 CSV not found. Downloading stock data...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


✅ Data downloaded and saved to reliance_stock.csv
         Date               Close
0         NaN         RELIANCE.NS
1  2015-01-01   196.6393585205078
2  2015-01-02  196.11891174316406
3  2015-01-05       193.970703125
4  2015-01-06   185.1674346923828
Date     object
Close    object
dtype: object
Date      object
Close    float64
dtype: object
📰 Avg News Sentiment (Last 3 Days): 0.07187921633887219


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 53ms/step - loss: 0.0399
Epoch 2/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - loss: 5.7035e-04
Epoch 3/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - loss: 4.8948e-04
Epoch 4/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - loss: 4.8158e-04
Epoch 5/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - loss: 5.1579e-04
Epoch 6/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 4.4631e-04
Epoch 7/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - loss: 5.1404e-04
Epoch 8/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 56ms/step - loss: 4.3726e-04
Epoch 9/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - loss: 3.9878e-04
Epoch 10/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - loss: 4.9223e-04
Epoch 11/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - loss: 4.1917e-04
Epoch 12/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - loss: 3.8555e-04
Epoch 13/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 61ms/step - loss: 4.0466e-04
Epoch 14/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 3.3320e-04
Epoch 15/20
78/78 ━